In [1]:
import numpy as np
import pandas as pd
import gzip
import glob
import sys
import os
from tqdm import tqdm

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('../utils/'))))
from utils.preprocess_utils import save_cache
data_path = '/data1/jyhwang/SR/'
file_name = 'Books.csv'
df = pd.read_csv(data_path + file_name)

In [2]:
df_view_org = df.copy()

In [3]:
df

,user_id,parent_asin,rating,timestamp
0,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1446304000,5.0,1441260345000
1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1564770672,5.0,1441260365000
2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1442450703,5.0,1523093714024
3,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1780671067,1.0,1611623223325
4,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1645671127,3.0,1612044209266
...,...,...,...,...
9488292,AGZ44L7OCCLE76RJOZ3VGKOEKLFQ,0743259823,4.0,1102914217000
9488293,AGZ44L7OCCLE76RJOZ3VGKOEKLFQ,0891230513,3.0,1102921345000
9488294,AGZ44L7OCCLE76RJOZ3VGKOEKLFQ,0891230483,3.0,1102921945000
9488295,AGZ44L7OCCLE76RJOZ3VGKOEKLFQ,0500016909,2.0,1103970985000


## Clean Data

In [4]:
df_view = df_view_org
# 2020. 01. 1 부터..
filtered_df = df_view[df_view.timestamp>1577808061000]
# 2017. 01. 0.1
1483196400000
# 2020. 0.1 01
1577808061000
#filtered_df = df_view

1577808061000

In [5]:
# User-Item Filtering
item_filter = (filtered_df['parent_asin'].value_counts() >= 5)
filtered_items = filtered_df['parent_asin'].value_counts()[item_filter].index

print("Item Filtering")
print(len(filtered_df))

user_filter = (filtered_df['user_id'].value_counts() >= 5)
filtered_users = filtered_df['user_id'].value_counts()[user_filter].index
filtered_df = filtered_df[filtered_df['user_id'].isin(filtered_users)]

print("User Filtering")
print(len(filtered_df))

# Extract interactions for model training (numpy.ndarray)
uit = filtered_df.loc[:, ['user_id', 'parent_asin', 'timestamp'] ].values
ust = filtered_df.loc[:, ['user_id', 'parent_asin', 'timestamp'] ].values

Item Filtering
1634099
User Filtering
1201417


In [6]:
from utils.preprocess_utils import map_feature_id
num_users,uid_map,uid_rmap = map_feature_id(uit[:,0], str, int)
num_items,iid_map,iid_rmap = map_feature_id(uit[:,1], str, int)
num_time,tid_map,tid_rmap = map_feature_id(sorted(uit[:,2]), str, int) #The mapped timeid should be sorted,
num_state,sid_map,sid_rmap = map_feature_id(ust[:,1], str, int)

In [7]:
from utils.preprocess_utils import build_seq_list
raw_sequences = build_seq_list(uit, uid_map, iid_map, tid_map)#
#raw_state_sequences= build_seq_list(ust, uid_map, sid_map, tid_map)
from utils.preprocess_utils import remove_redundant_items
item_sequences = remove_redundant_items(raw_sequences, raw_sequences)
#state_sequences= remove_redundant_items(raw_state_sequences, raw_state_sequences)

In [8]:
from utils.preprocess_utils import split_seqs
'''
For debugging code.

item_sequences = [[1, 2, 3, 4, 5, 6, 7],
                  [1, 3, 5, 7, 9, 11,1],
                  [2, 4, 2, 6, 8, 9, 4]]

'''
train_seqs, valid_test_seqs = split_seqs(item_sequences,0.8)
valid_seqs, test_seqs = split_seqs(valid_test_seqs,0.5)

from utils.preprocess_utils import extract_valid_seqs
train_valid_seqs = list(map(lambda seq: seq[0] + seq[1],
                            zip(train_seqs,valid_seqs)))

valid_seqs, valid_items = extract_valid_seqs(train_seqs, valid_seqs)
test_seqs, test_items = extract_valid_seqs(train_valid_seqs, test_seqs)
'''
For debugging code.

print(train_seqs)
print(train_valid_seqs)
print("Validation ")
print(valid_seqs)
print(valid_items)
print("Testing")
print(test_seqs)
print(test_items)
'''

from utils.preprocess_utils import generate_pairs_from_sequences
from scipy.sparse import csr_matrix
valid_seen_pairs = np.array(list(generate_pairs_from_sequences(valid_seqs)))
valid_seen_vals = np.ones(len(valid_seen_pairs))
valid_seen_rows = valid_seen_pairs[:,0]
valid_seen_cols = valid_seen_pairs[:,1]
valid_seen_mat = csr_matrix((valid_seen_vals,
                            (valid_seen_rows,
                             valid_seen_cols)), shape = (num_users,num_items))


valid_unseen_pairs = np.array(list(generate_pairs_from_sequences(valid_items)))
valid_unseen_vals = np.ones(len(valid_unseen_pairs))
valid_unseen_rows = valid_unseen_pairs[:,0]
valid_unseen_cols = valid_unseen_pairs[:,1]
valid_unseen_mat = csr_matrix((valid_unseen_vals,
                            (valid_unseen_rows,
                             valid_unseen_cols)), shape = (num_users,num_items))


test_seen_pairs = np.array(list(generate_pairs_from_sequences(test_seqs)))
test_seen_vals = np.ones(len(test_seen_pairs))
test_seen_rows = test_seen_pairs[:,0]
test_seen_cols = test_seen_pairs[:,1]
test_seen_mat = csr_matrix((test_seen_vals,
                            (test_seen_rows,
                             test_seen_cols)), shape = (num_users,num_items))


test_unseen_pairs = np.array(list(generate_pairs_from_sequences(test_items)))
test_unseen_vals = np.ones(len(test_unseen_pairs))
test_unseen_rows = test_unseen_pairs[:,0]
test_unseen_cols = test_unseen_pairs[:,1]
test_unseen_mat = csr_matrix((test_unseen_vals,
                             (test_unseen_rows,
                             test_unseen_cols)), shape = (num_users,num_items))

In [9]:
sorted_pos_lists = [sorted(items) for items in train_seqs] # For efficient negative sampling in training time.

In [10]:
print("num_users:%s num_items:%s"%(num_users,num_items))

num_users:97238 num_items:195053


In [11]:
from utils.preprocess_utils import save_cache
objs = {
    "num_users": num_users,
    "num_items": num_items,
    "train_seqs": train_seqs,
    "valid_seqs": valid_seqs,
    "test_seqs": test_seqs,
    "valid_seen_mat": valid_seen_mat,
    "valid_gt_mat": valid_unseen_mat,
    "test_seen_mat": test_seen_mat,
    "test_gt_mat": test_unseen_mat,
    "sorted_pos_lists": sorted_pos_lists
}
save_cache(data_path,'Books_cache',objs)

In [12]:
valid_unseen_pairs

array([[    1,     9],
       [    2,    30],
       [    3,    63],
       ...,
       [42766, 36943],
       [42767, 52222],
       [42768, 51575]])

In [15]:
len(valid_seqs)

16

In [16]:
len(train_seqs)

16

In [17]:
num_users

16

In [14]:
df

,user_id,parent_asin,rating,timestamp
0,AFWHJ6O3PV4JC7PVOJH6CPULO2KQ,B000002AGY,5.0,1358286606000
1,AFWHJ6O3PV4JC7PVOJH6CPULO2KQ,B002HMHR7S,4.0,1402778050000
2,AFWHJ6O3PV4JC7PVOJH6CPULO2KQ,B0000062P5,5.0,1504898965457
3,AFWHJ6O3PV4JC7PVOJH6CPULO2KQ,B00004NKAK,5.0,1524768111415
4,AFWHJ6O3PV4JC7PVOJH6CPULO2KQ,B07Z76Y18X,5.0,1576100171173
...,...,...,...,...
1552759,AHHXVAEKAYJGVWACMYCSBQUND7TQ,B000002HQ3,5.0,1004132347000
1552760,AHHXVAEKAYJGVWACMYCSBQUND7TQ,B000002KHH,5.0,1011831232000
1552761,AHHXVAEKAYJGVWACMYCSBQUND7TQ,B00004Y6O6,4.0,1012932465000
1552762,AHHXVAEKAYJGVWACMYCSBQUND7TQ,B00000AGAS,5.0,1020810521000


In [ ]:
#문제의 코드
import numpy as np
import torch
import time
import copy
from tqdm import tqdm
from collections import defaultdict

def _get_recall(batch_hits, num_tests, cond):

    return batch_hits.sum(1,keepdims=True)[cond]/(num_tests)[cond]

def _get_ndcg(batch_hits, num_tests, cond):
    
    topk = batch_hits.size(1)
    denominator = torch.log2(1+torch.arange(1,topk+1)).unsqueeze(0).cuda()
    DCG_K = (batch_hits/denominator)[cond]
    
    IDCG_K = torch.cumsum((1./denominator),1).flatten()
    
    IDCG_I = num_tests[cond].flatten().long() -1
    IDCG_I[IDCG_I>=topk] = topk-1
    
    norm = IDCG_K[IDCG_I].unsqueeze(1)
    
    NDCG_K = (DCG_K).sum(1,keepdims=True)/norm
    
    return NDCG_K
    
def _get_rr(batch_hits, cond):
    denominator = (1.0 / torch.arange(1, batch_hits.size(1) + 1)).cuda()
    denominator = denominator.unsqueeze(0)

    # rr is reciprocal rank
    rr = ((batch_hits >= 1.0) * denominator * batch_hits)[cond]
    rr = torch.max(rr, 1).values

    return rr
    


def evaluate_topk(model, seq_list, dataset, scenario = 'valid', topk = 20):

    '''
     evaluate topk ranking performance
     (currently, only recall is implemented)
     
     :param model: recommender system
     :param list seq_list: user list for test
     :param dataset: dataset
     :param int topk: # of items to show
     "
    '''
    
    model.eval()
    if scenario == 'valid':
        coo = dataset.valid_seen_mat.tocoo()       
        ind = torch.LongTensor([coo.row.tolist(), coo.col.tolist()])
        val = torch.LongTensor(coo.data.astype(np.int32))
        seen_mat = torch.sparse_coo_tensor(ind, val, (dataset.num_seqs, dataset.num_items)).cuda()
        
        coo = dataset.valid_gt_mat.tocoo()
        ind = torch.LongTensor([coo.row.tolist(), coo.col.tolist()])
        val = torch.LongTensor(coo.data.astype(np.int32))
        gt_mat = torch.sparse_coo_tensor(ind, val, (dataset.num_seqs, dataset.num_items)).cuda()
        
        last_subseqs = torch.LongTensor(dataset.valid_last_subseqs[:,1:]).cuda()
    if scenario == 'test':
        coo = dataset.test_seen_mat.tocoo()
        ind = torch.LongTensor([coo.row.tolist(), coo.col.tolist()])
        val = torch.LongTensor(coo.data.astype(np.int32))
        seen_mat = torch.sparse_coo_tensor(ind, val, (dataset.num_seqs, dataset.num_items)).cuda()
        
        coo = dataset.test_gt_mat.tocoo()
        ind = torch.LongTensor([coo.row.tolist(), coo.col.tolist()])
        val = torch.LongTensor(coo.data.astype(np.int32))
        gt_mat = torch.sparse_coo_tensor(ind, val, (dataset.num_seqs, dataset.num_items)).cuda()
        #check last subseqs
        last_subseqs = torch.LongTensor(dataset.test_last_subseqs[:,1:]).cuda()
    
    #prepare sequences
    seq_indices = torch.split(torch.LongTensor(seq_list),256)
    all_item_indices = torch.arange(dataset.num_items).unsqueeze(1).cuda()
    
    recall = torch.cuda.DoubleTensor()
    ndcg = torch.cuda.DoubleTensor()
    mrr = torch.cuda.DoubleTensor()
    
    with torch.no_grad():
        for i, batch_seq_indices in enumerate(tqdm(seq_indices, desc="Evaluation", disable = True)):
            batch_seq_indices = batch_seq_indices.cuda()
            batch_scores = torch.zeros( (len(batch_seq_indices), dataset.num_items)).cuda()
            predicted_scores = model(batch_seq_indices,
                                     last_subseqs[batch_seq_indices],
                                     all_item_indices, # NX1
                                     pred_opt='eval')
            #print(predicted_scores)
            #breakpoint()
            batch_seen_array = torch.index_select(seen_mat, 0, batch_seq_indices).to_dense()
            batch_scores.unsqueeze(0)[:, batch_seen_array > 0 ] = -1000.0 # ignore seen items
            batch_topk = torch.topk(batch_scores, k = topk, dim=1, sorted=True).indices
            
            batch_gt_array = torch.index_select(gt_mat, 0, batch_seq_indices).to_dense().long()
            
            num_tests = batch_gt_array.sum(1,keepdims=True)
            batch_hits = torch.gather(batch_gt_array, 1, batch_topk)

            #select validation user
            cond = (num_tests > 0.).flatten()
            
            batch_recall = _get_recall(batch_hits, num_tests, cond)
            recall = torch.cat((recall, batch_recall),0)
            
            batch_ndcg = _get_ndcg(batch_hits, num_tests, cond)
            ndcg = torch.cat((ndcg, batch_ndcg),0)
            
            batch_mrr = _get_rr(batch_hits, cond)
            mrr = torch.cat((mrr, batch_mrr.type('torch.DoubleTensor').cuda()), 0)

    return {"recall":recall.mean(),"ndcg":ndcg.mean(), "mrr":mrr.mean() }